# **자기주도PJT - 심화과제**

**Req.7. 다음의 과정을 따라 advanced.ipynb에 VoxCeleb Dataset을 사용해 결과를 출력 해보세요.**
1. https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw 에서 동영상 파일 (예, 04.mp4) 하나를 다운로드 받아 구글 드라이브 first_motion_order_model 폴더에 복사해 놓습니다.
2. https://drive.google.com/drive/folders/1PyQJmkdCsAkOYwUyaj_l-l0as-iLDgeH 에서 다음의 Dataset 파일을 다운로드 받아 구글 드라이브 first_motion_order_model 폴더에 복사해 놓습니다.
  - vox-cpk.pth.tar
3. 휴대폰으로 셀카를 찍어 my_face.png 라는 파일명으로 구귿 드라이브 first_motion_order_model 폴더에 올려 놓습니다. (정사각형 형태로 촬영을 추천 합니다.)


In [1]:
# 구글 드라이브를 /gdrive에 마운트 합니다.
from google.colab import drive
#drive.flush_and_unmount()  # 파일목록 갱신이 안될때는 이 부분 주석처리를 해제하고 실행하시기 바랍니다.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
# 마운트된 드라이브의 first_order_motion_model 디렉토리의 파일목록을 불러옵니다.
!ls -l /gdrive/MyDrive/Colab_Notebooks/colab/first_motion_order_model

total 1447102
-rw------- 1 root root    170054 Jun 19 13:52  10.mp4
-rw------- 1 root root     36604 Dec 22 10:43 '증명사진(211222)-저용량-저용량.jpg'
-rw------- 1 root root    942794 Jun 19 14:08  91IiZsMxf9S.mp4
-rw------- 1 root root 750842814 Jun 19 14:10  fashion.pth.tar
-rw------- 1 root root    252506 Jun 19 14:56  m1.PNG
-rw------- 1 root root    327731 Jun 19 14:59  m2.PNG
-rw------- 1 root root    314008 Jun 19 14:59  m3.PNG
-rw------- 1 root root    176228 Jun 19 13:39  statue-02.png
-rw------- 1 root root 728766691 Jun 19 13:40  vox-cpk.pth.tar


In [3]:
# Aliaksandr Siarohin 의 github에서 First Order Model을 clone 합니다.
!git clone https://github.com/AliaksandrSiarohin/first-order-model

fatal: destination path 'first-order-model' already exists and is not an empty directory.


In [4]:
# 디렉토리를 변경합니다.

In [5]:
cd /content/first-order-model

/content/first-order-model


In [6]:
# 현재 디렉토리의 내용을 보여줍니다.
!ls -l

total 1284
-rw-r--r-- 1 root root    4061 Jun 19 13:44 animate.py
-rw-r--r-- 1 root root   12547 Jun 19 13:44 augmentation.py
drwxr-xr-x 2 root root    4096 Jun 19 13:44 config
-rw-r--r-- 1 root root    5876 Jun 19 13:44 crop-video.py
drwxr-xr-x 3 root root    4096 Jun 19 13:44 data
-rw-r--r-- 1 root root   21284 Jun 19 13:44 demo.ipynb
-rw-r--r-- 1 root root    6730 Jun 19 13:44 demo.py
-rw-r--r-- 1 root root     502 Jun 19 13:44 Dockerfile
drwxr-xr-x 8 root root    4096 Jun 19 13:51 first-order-model
-rw-r--r-- 1 root root    7027 Jun 19 13:44 frames_dataset.py
-rw-r--r-- 1 root root   18868 Jun 19 13:44 LICENSE.md
-rw-r--r-- 1 root root    8282 Jun 19 13:44 logger.py
drwxr-xr-x 3 root root    4096 Jun 19 13:49 modules
-rw-r--r-- 1 root root 1143882 Jun 19 13:44 old_demo.ipynb
drwxr-xr-x 2 root root    4096 Jun 19 13:49 __pycache__
-rw-r--r-- 1 root root    8727 Jun 19 13:44 README.md
-rw-r--r-- 1 root root    2772 Jun 19 13:44 reconstruction.py
-rw-r--r-- 1 root root     210 Jun 19 

In [7]:
# 필요한 패키지들을 import 합니다.
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte


In [10]:
# 소스 이미지
source_image = imageio.imread('/gdrive/MyDrive/Colab_Notebooks/colab/first_motion_order_model/증명사진(211222)-저용량-저용량.jpg')
# 소스 비디오
reader = imageio.get_reader('/gdrive/MyDrive/Colab_Notebooks/colab/first_motion_order_model/10.mp4')
# 이미지와 비디오크기를 256x256 픽셀로 변경 합니다
source_image = resize(source_image, (256,256))[...,:3]
# frames per second 1초당 표시할 프레임수를 가져옵니다
fps = reader.get_meta_data()['fps']

In [11]:
#driving_video를 초기화 하고 소스비디오의 각 프레임을 driving_video에 추가 합니다.
driving_video = []
try:
  for im in reader:
    driving_video.append(im)
except RuntimeError:
  pass
reader.close()

In [12]:
# driving_video를 256x256 으로 리사이즈 합니다.
driving_video = [resize(frame, (256,256))[...,:3] for frame in driving_video]


In [8]:
# 소스이미지와 기존 비디오를 합성해주는 display 함수
def display(source, driving, generated=None):
  fig = plt.figure(figsize = (8+4 *(generated is not None), 6))
  ims = []
  for i in range(len(driving)):
    cols = [source]
    cols.append(driving[i])
    if generated is not None:
      cols.append(generated[i])
    im = plt.imshow(np.concatenate(cols, axis=1), animated = True)
    plt.axis('off')
    ims.append([im])
  ani = animation.ArtistAnimation(fig, ims, interval = 50, repeat_delay= 1000)
  plt.close()
  return ani

In [14]:
# display() 함수를 사용하여 HTML5 비디오 형태로 출력합니다.
HTML(display(source_image, driving_video).to_html5_video())

In [15]:
# VoxCeleb 모델의 체크포인트 불러오기 (700MB 파일이라 오래 걸립니다.)
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml',
                                          checkpoint_path = '/gdrive/MyDrive/Colab_Notebooks/colab/first_motion_order_model/vox-cpk.pth.tar')

In [16]:
# make_animation() 함수를 사용하여 소스이미지, 소스비디오를 입력으로 predictions를 생성합니다.
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative = True)

# generated.mp4 파일로 생성된 결과를 저장
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

# display 함수를 사용해 로 출력합니다.
HTML(display(source_image, driving_video, predictions).to_html5_video())


100%|██████████| 444/444 [00:16<00:00, 27.53it/s]






---




**Req.8. 다음의 과정을 따라 advanced.ipynb에 Fashion-Videos Dataset을 사용해 결과를 출력 해보세요.**
1.	https://drive.google.com/drive/folders/1PyQJmkdCsAkOYwUyaj_l-l0as-iLDgeH 에서 다음의 Dataset 파일을 다운로드 받아 구글 드라이브 first_motion_order_model 폴더에 복사해 놓습니다.
  -	fashion.pth.tar
2.	https://vision.cs.ubc.ca/datasets/fashion/resources/fashion_dataset/fashion_train.txt 에서 4개의 mp4 파일을 다운로드 받습니다. 
3.	다운로드 받은 mp4파일을 전체화면으로 재생시킨 후 바로 Pause 버튼을 눌러 모델이 똑바로 앞을 보고 있는 장면에서 멈춘 후 컴퓨터의 화면 캡춰 프로그램을 사용해 동영상 화면을 캡춰해 m1,png, m2,png, mp3.png 등 3개의 정지화면을 얻은 후 구글드라이브에 복사해 놓습니다.
4.	나머지 1개의 동영상 혹은 해당 동영상과 비슷한 구도와 포즈의 전신 샷을 촬영하여 req8_source.mp4라는 이름으로 구글드라이브에 복사해 놓습니다.


In [9]:
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

In [10]:
# 3개의 이미지와 1개의 동영상을 불러 옵니다.
# 소스 이미지
source_image1 = imageio.imread('/gdrive/MyDrive/Colab_Notebooks/colab/first_motion_order_model/m1.PNG')
source_image2 = imageio.imread('/gdrive/MyDrive/Colab_Notebooks/colab/first_motion_order_model/m2.PNG')
source_image3 = imageio.imread('/gdrive/MyDrive/Colab_Notebooks/colab/first_motion_order_model/m3.PNG')
# 소스 비디오
reader = imageio.get_reader('/gdrive/MyDrive/Colab_Notebooks/colab/first_motion_order_model/91IiZsMxf9S.mp4')

fps = reader.get_meta_data()['fps']

In [11]:
# 이미지와 비디오크기를 256x256 픽셀로 변경 합니다
source_image1 = resize(source_image1, (256,256))[...,:3]
source_image2 = resize(source_image2, (256,256))[...,:3]
source_image3 = resize(source_image3, (256,256))[...,:3]

# reader = resize(reader, (256,256))[...,:3]

In [12]:
# load_checkpoints 함수를 사용해 이전에 학습해둔 fashion.pht.tar을 을 불러 옵니다.
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml',
                                          checkpoint_path = '/gdrive/MyDrive/Colab_Notebooks/colab/first_motion_order_model/fashion.pth.tar')

In [13]:
driving_video = []
try:
  for im in reader:
    driving_video.append(im)
except RuntimeError:
  pass
reader.close()

driving_video = [resize(frame, (256,256))[...,:3] for frame in driving_video]

In [14]:
# 첫번째 모델의 사진과 Driving Video로 첫번째 모델의 움직이는 비디오를 만들고 재생합니다.

# make_animation() 함수를 사용하여 소스이미지, 소스비디오를 입력으로 predictions를 생성합니다.
predictions = make_animation(source_image1, driving_video, generator, kp_detector, relative = True)

# generated.mp4 파일로 생성된 결과를 저장
imageio.mimsave('../generated1.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

# display 함수를 사용해 로 출력합니다.
HTML(display(source_image1, driving_video, predictions).to_html5_video())

100%|██████████| 353/353 [00:12<00:00, 28.62it/s]


In [15]:
# 두번째 모델의 사진과 Driving Video로 두번째 모델의 움직이는 비디오를 만들고 재생합니다.
# make_animation() 함수를 사용하여 소스이미지, 소스비디오를 입력으로 predictions를 생성합니다.
predictions = make_animation(source_image2, driving_video, generator, kp_detector, relative = True)

# generated.mp4 파일로 생성된 결과를 저장
imageio.mimsave('../generated2.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

# display 함수를 사용해 로 출력합니다.
HTML(display(source_image2, driving_video, predictions).to_html5_video())

100%|██████████| 353/353 [00:12<00:00, 28.17it/s]


In [16]:
# 세번째 모델의 사진과 Driving Video로 세번째 모델의 움직이는 비디오를 만들고 재생합니다.
# make_animation() 함수를 사용하여 소스이미지, 소스비디오를 입력으로 predictions를 생성합니다.
predictions = make_animation(source_image3, driving_video, generator, kp_detector, relative = True)

# generated.mp4 파일로 생성된 결과를 저장
imageio.mimsave('../generated3.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

# display 함수를 사용해 로 출력합니다.
HTML(display(source_image3, driving_video, predictions).to_html5_video())

100%|██████████| 353/353 [00:13<00:00, 26.27it/s]
